In [ ]:
from selenium.webdriver import Firefox
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains

from bs4 import BeautifulSoup
import time

## 1. Requesting data

### 1.1 get seller URLs

In [ ]:
driver = Firefox(executable_path=r"geckodriver.exe")

In [ ]:
url = "https://www.fiverr.com/?source=top_nav"
driver.get(url)

In [ ]:
# search for key words
search_term = 'data science' 
driver.find_elements_by_xpath(".//input[@type='search']")[1].send_keys(search_term, Keys.ENTER)

In [ ]:
# check for denied access
soup = BeautifulSoup(driver.page_source, "lxml")
soup.title.text == 'Your Access To This Website Has Been Blocked'

In [ ]:
# test for sign-up pop up
pop_up = driver.find_elements_by_class_name('sign-up-form')
action = ActionChains(driver)
if len(pop_up)>0:
    action.move_to_element_with_offset(pop_up[0], 700, 0)
    action.click()
    action.perform()

In [ ]:
# no. of search results
no_of_services = driver.find_element_by_class_name('number-of-results').text
no_of_services

In [ ]:
# scroll to botom page
js_script = "window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;"
lenOfPage = driver.execute_script(js_script)
match=False
while(match==False):
    lastCount = lenOfPage
    lenOfPage = driver.execute_script(js_script)
    if lastCount==lenOfPage:
        match=True

In [ ]:
# click next page
right_arrow = driver.find_elements_by_class_name('pagination-arrows')[-1]
right_arrow.click()

In [ ]:
right_arrow = driver.find_elements_by_class_name('pagination-arrows')[-1]
right_arrow.click()

In [ ]:
# get seller information
soup = BeautifulSoup(driver.page_source, "lxml")
gigs = soup.findAll('div', {'class': 'gig-card-layout'})

for gig in gigs[:1]:
    seller_url = "https://www.fiverr.com" + gig.h3.a['href']
    seller_name = gig.find('div', {'class': 'seller-name'}).text
    print(seller_name, url)    

In [ ]:
# iterate over pages
scroll = True

while scroll:
    # scroll down
    js_script = "window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;"
    lenOfPage = driver.execute_script(js_script)
    match=False
    while(match==False):
        lastCount = lenOfPage
        lenOfPage = driver.execute_script(js_script)
        if lastCount==lenOfPage:
            match=True
            
    time.sleep(1)
            
    # click next page
    arrows = driver.find_elements_by_class_name('pagination-arrows')
    if len(arrows) == 2:
        try:
            arrows[-1].click()
        except:
            scroll = False
            
    else:
        scroll = False
        
    time.sleep(1)
        
    # get seller information
    soup = BeautifulSoup(driver.page_source, "lxml")
    gigs = soup.findAll('div', {'class': 'gig-card-layout'})
    
    # print first gig info
    for gig in gigs[:1]:
        seller_url = "https://www.fiverr.com" + gig.h3.a['href']
        seller_name = gig.find('div', {'class': 'seller-name'}).text
        print(seller_name, url)

In [ ]:
driver.quit()